In [1]:
import os
os.chdir('../')
%pwd

'/home/paladin/Downloads/Bixi-OD-Matrix-Prediction/Bixi-OD-Matrix-Prediction'

In [2]:
import sys
import numpy as np
import keras
from pathlib import Path
import pickle
from timeseriesPredictor.exception import CustomException

2023-08-16 11:09:52.597410: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-16 11:09:52.779961: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-16 11:09:52.781829: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 11:09:54.053421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def load_pickle_file(path: Path):
    file= open(path, 'rb')
    obj = pickle.load(file)
    return obj

data = load_pickle_file('artifacts/data_transformation/tensor/train.pkl')
data.shape

(635, 100, 64, 1)

In [4]:
dat = data.reshape(data.shape[0],100*64)

In [5]:
def split_sequence(sequence, lag):
        '''
        This function splits a given univariate sequence into
        multiple samples where each sample has a specified number
        of time steps and the output is a single time step.
        param new_input: If True it is used for predicting new input
        '''
        try:
                
            X, y = list(), list()
            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix = i + lag
                # check if we are beyond the sequence               
                if end_ix > len(sequence)-1:
                    break
            # gather input and output parts of the pattern
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return np.array(X), np.array(y)
    
        except Exception as e:
            raise CustomException(e, sys)
    
def convert_to_supervised(dat, lag):
        '''
        This function takes a 2D sequennce, scales the array and splits
        a given multivariate sequence into multiple samples where each sample has a specified number
        of time steps. It returns multiple time steps as well as the scaler.
        param df (DataFrame): Bike sharing demand for each community over time
        param lag (int): History length or time lag
        '''
        
        try:
            if isinstance(dat, np.ndarray):
                pass
            else:
                dat = dat.values
            
            m, n = dat.shape
            # e.g., if lag = 7, BIXI demand of past 7*15 minutes
            X = np.zeros((m-lag,lag, n))
            Y = np.zeros((m-lag,n))

            for i in range(0,n):
                x, y = split_sequence(dat[:,i], lag)
                X[:,:,i] = x
                Y[:,i] = y
            return X, Y
    
        except Exception as e:
            raise CustomException(e, sys)   

In [6]:
lag = 7
X, y = convert_to_supervised(dat, lag)
train_x = X.reshape(X.shape[0], 7, 100,64, 1)
train_y = y.reshape(y.shape[0], 100, 64, 1)
X.shape

(628, 7, 6400)

In [7]:
y.shape

(628, 6400)

In [8]:
print(train_x.shape)
print(train_y.shape)

(628, 7, 100, 64, 1)
(628, 100, 64, 1)


In [21]:
# define encoder
input = keras.Input(shape= (7, 100, 64, 1))
encoder = keras.layers.ConvLSTM2D(filters=64, kernel_size=(3,3), activation='relu', return_sequences=True)(input)
encoder = keras.layers.BatchNormalization()(encoder)
encoder = keras.layers.ConvLSTM2D(filters=32, kernel_size=(3,3), activation='relu', return_sequences=True)(encoder)
encoder = keras.layers.BatchNormalization()(encoder)
encoder = keras.layers.Flatten()(encoder)
# define predict decoder
decoder= keras.layers.RepeatVector(100*64)(encoder)
decoder= keras.layers.LSTM(64, activation='relu', return_sequences=True)(decoder)
decoder = keras.layers.BatchNormalization()(decoder)
decoder= keras.layers.LSTM(32, activation='relu', return_sequences=True)(decoder)
decoder = keras.layers.BatchNormalization()(decoder)
decoder = keras.layers.TimeDistributed(keras.layers.Dense(1, activation='relu'))(decoder)
decoder = keras.layers.Reshape(target_shape=(100, 64, 1))(decoder)
decoder = keras.layers.Convolution2D(filters=1, kernel_size=(3, 3), activation='linear', padding='same')(decoder) 

model = keras.Model(inputs= input, outputs=decoder)
model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss = 'mean_squared_error' ,
            metrics=['MSE']           
                           )

model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 7, 100, 64, 1)]   0         
                                                                 
 conv_lstm2d_24 (ConvLSTM2D  (None, 7, 98, 62, 64)     150016    
 )                                                               
                                                                 
 conv_lstm2d_25 (ConvLSTM2D  (None, 7, 96, 60, 32)     110720    
 )                                                               
                                                                 
 flatten_12 (Flatten)        (None, 1290240)           0         
                                                                 
 repeat_vector_12 (RepeatVe  (None, 6400, 1290240)     0         
 ctor)                                                           
                                                           

In [10]:
history = model.fit(train_x, train_y, epochs=1, batch_size=1, verbose=1)      

2023-08-16 10:16:41.919380: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1269760000 exceeds 10% of free system memory.
2023-08-16 10:16:42.085583: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1269760000 exceeds 10% of free system memory.
